# Traveling Salesman Problem (TSP): Context + Two Implementations

**What you'll find here**
1. Background and context on the Traveling Salesman Problem (TSP)
2. A **dependency-free** TSP solver in pure Python using **2-opt local search**
3. A **Mixed-Integer Linear Programming (MILP)** formulation using **PuLP**

**How to use this notebook**
- Run Section 2 for a no-dependency solution that works anywhere Python runs.
- Run Section 3 if you have (or can install) `pulp` to see an exact MILP approach for small-to-medium instances.


## 1) Background: What is the TSP?
The **Traveling Salesman Problem (TSP)** asks: given a set of cities and distances between them, find the **shortest possible tour** that visits each city **exactly once** and returns to the starting city.

**Why it's important**
- It captures core ideas in **combinatorial optimization**.
- It's **NP-hard**, so exact solutions scale poorly; heuristics and metaheuristics are key in practice.
- TSP appears in logistics, routing drones/robots, circuit board design, DNA sequencing (as a model), and more.

**Growth of possibilities**
For `n` cities, the number of distinct tours is roughly `(n-1)!/2` (for undirected symmetric distances). Even `n=20` is astronomically large, so brute force is infeasible.

**Two families of approaches**
- **Exact**: MILP/ILP with branch & bound, cutting planes, dynamic programming (e.g., Held–Karp). Guarantees optimality but limited scale.
- **Heuristics/metaheuristics**: nearest neighbor, 2-opt/3-opt, simulated annealing, genetic algorithms, ant colony, etc. Scales better; no optimality guarantee.


## 2) Dependency-free TSP (2-opt local search)
Below is a **single pure-Python** TSP implementation:
- Builds a small set of 2D points
- Computes a distance matrix
- Constructs an initial tour via **nearest neighbor**
- Improves the tour with **2-opt** until no improvements remain

This is great for teaching and small demos (tens to hundreds of cities).

In [7]:
# --- Dependency-free TSP with 2-opt ---
from typing import List, Tuple
import math, random

Point = Tuple[float, float]

def euclid(p: Point, q: Point) -> float:
    return math.hypot(p[0]-q[0], p[1]-q[1])

def build_dist_matrix(pts: List[Point]) -> List[List[float]]:
    n = len(pts)
    D = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            d = euclid(pts[i], pts[j])
            D[i][j] = D[j][i] = d
    return D

def tour_length(tour: List[int], D: List[List[float]]) -> float:
    n = len(tour)
    s = 0.0
    for k in range(n):
        i = tour[k]
        j = tour[(k+1) % n]
        s += D[i][j]
    return s

def nearest_neighbor(D: List[List[float]], start: int = 0) -> List[int]:
    n = len(D)
    unv = set(range(n))
    tour = [start]
    unv.remove(start)
    cur = start
    while unv:
        nxt = min(unv, key=lambda j: D[cur][j])
        tour.append(nxt)
        unv.remove(nxt)
        cur = nxt
    return tour

def two_opt_once(tour: List[int], D: List[List[float]]):
    n = len(tour)
    best_gain, best = 0.0, None
    for i in range(n-1):
        for j in range(i+2, n if i>0 else n-1):
            a, b = tour[i], tour[(i+1)%n]
            c, d = tour[j], tour[(j+1)%n]
            old = D[a][b] + D[c][d]
            new = D[a][c] + D[b][d]
            gain = old - new
            if gain > 1e-12 and gain > best_gain:
                best_gain = gain
                best = (i, j)
    if best is not None:
        i, j = best[0]+1, best[1]
        tour[i:j+1] = reversed(tour[i:j+1])
        return tour, True
    return tour, False

def two_opt(tour: List[int], D: List[List[float]]):
    improved = True
    while improved:
        tour, improved = two_opt_once(tour, D)
    return tour

def solve_tsp_dependency_free(points: List[Point], seed: int = 42, restarts: int = 20):
    random.seed(seed)
    D = build_dist_matrix(points)
    n = len(points)
    best_tour, best_len = None, float('inf')
    # a few NN starts
    for s in range(min(n, 5)):
        t0 = nearest_neighbor(D, s)
        t1 = two_opt(t0[:], D)
        L = tour_length(t1, D)
        if L < best_len:
            best_tour, best_len = t1[:], L
    # random restarts
    for _ in range(restarts):
        t0 = list(range(n))
        random.shuffle(t0)
        t1 = two_opt(t0, D)
        L = tour_length(t1, D)
        if L < best_len:
            best_tour, best_len = t1[:], L
    return best_tour, best_len, D

# Example points (edit freely)
CITIES = [
    (0,0),(1,5),(5,2),(6,6),(8,3),(2,1),(7,8),(3,7),(9,4),(4,3)
]

tour, length, D = solve_tsp_dependency_free(CITIES, restarts=50)
print('Best tour (indices):', tour)
print('Best length:', round(length, 4))
print('Readable route:')
for idx in tour:
    print(f'  {idx}: {CITIES[idx]}')
print('  back to start:', CITIES[tour[0]])

Best tour (indices): [2, 9, 5, 0, 1, 7, 3, 6, 8, 4]
Best length: 28.8531
Readable route:
  2: (5, 2)
  9: (4, 3)
  5: (2, 1)
  0: (0, 0)
  1: (1, 5)
  7: (3, 7)
  3: (6, 6)
  6: (7, 8)
  8: (9, 4)
  4: (8, 3)
  back to start: (5, 2)


## 🚗 Traveling Salesman Problem (TSP) — Undirected Formulation (No MTZ)

In previous examples, we used **linear programming (LP)** and **mixed-integer programming (MIP)** to model optimization problems such as fertilizer cost and field selection.  
Now we revisit one of the most famous NP-hard problems: the **Traveling Salesman Problem (TSP)** — but this time, using a **pure undirected LP formulation**.

### 🧭 Problem Overview
Given a set of cities and distances between each pair, the goal is to find the **shortest possible tour** that:
- Visits every city exactly once, and  
- Returns to the starting city.

### 🧮 Formulation (Undirected)
We create one **binary variable** for each undirected edge:

\[
x_{ij} =
\begin{cases}
1, & \text{if edge between cities } i \text{ and } j \text{ is used in the tour,}\\
0, & \text{otherwise.}
\end{cases}
\]

**Objective:**
\[
\text{Minimize } \sum_{i<j} d_{ij} \, x_{ij}
\]

**Constraints:**
1. **Degree constraints:**  
   Each city must connect to exactly two other cities  
   \[
   \sum_{i<k} x_{ik} + \sum_{k<j} x_{kj} = 2, \quad \forall k
   \]
2. **Subtour elimination constraints:**  
   To prevent disconnected loops, for any subset of cities \( S \) smaller than \( n \):
   \[
   \sum_{i<j,\, i,j \in S} x_{ij} \le |S| - 1
   \]

These constraints ensure that the final solution forms a **single Hamiltonian cycle** through all cities.

---

### 🧠 Why “No MTZ”?
The **MTZ formulation** adds extra variables (\(u_i\)) to eliminate subtours using directed constraints.  
Here, we take a more direct, *undirected* approach:
- We start with only the degree constraints.
- After solving, if the solution includes multiple smaller cycles (subtours), we detect them.
- We then **add additional constraints (cuts)** dynamically to remove those subtours.
- This process repeats until there’s exactly **one connected tour** — the optimal solution.

This iterative approach is conceptually simpler and shows how solvers can *refine* models by adding constraints dynamically, similar to **cutting-plane methods** used in modern optimization.

---

### ⚙️ How It Works in Practice
The implementation below:
1. Builds the initial LP model with degree constraints.  
2. Solves it to find the shortest set of edges satisfying those constraints.  
3. Detects subtours (disconnected loops).  
4. Adds a constraint (“cut”) forbidding each subtour.  
5. Repeats until one full tour remains.

When finished, we have the **optimal undirected TSP tour** and its total length — all without using the MTZ variables.

---

### 🎯 Learning Connections
| Concept | Link to Earlier Topics |
|----------|------------------------|
| Linear constraints | Same LP framework as before (objective + constraints) |
| Binary decisions | Same as field selection (0–1 choices) |
| Iterative refinement | Similar to penalty updates or constraint tightening |
| NP-hardness | Another example of combinatorial optimization using LP tools |

---

Next, we’ll run the **undirected TSP model with subtour elimination** on our set of 10 cities.

In [8]:
# If PuLP is not installed locally, uncomment the next line:
# !pip install pulp

In [9]:
from typing import List, Tuple, Dict, Set
import math
import pulp

Point = Tuple[float, float]

def euclid(p: Point, q: Point) -> float:
    return math.hypot(p[0]-q[0], p[1]-q[1])

def tsp_undirected_subtour_elimination(points: List[Point], msg: bool = False, tol: float = 1e-6):
    """
    Pure undirected TSP (no MTZ).
    Uses iterative subtour elimination cuts:
      minimize sum_{i<j} d_ij x_ij
      s.t. degree(k) = 2  for all k
      and for each subtour S: sum_{i<j, i,j in S} x_ij <= |S| - 1
    """
    n = len(points)
    D = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            d = euclid(points[i], points[j])
            D[i][j] = D[j][i] = d

    # Build base model
    prob = pulp.LpProblem("TSP_Undirected_SEC", pulp.LpMinimize)

    # One binary variable per undirected edge (i<j)
    x = {(i, j): pulp.LpVariable(f"x_{i}_{j}", lowBound=0, upBound=1, cat="Binary")
         for i in range(n) for j in range(i+1, n)}

    # Objective
    prob += pulp.lpSum(D[i][j] * x[(i, j)] for i in range(n) for j in range(i+1, n)), "Obj"

    # Degree constraints: each node has degree 2
    for k in range(n):
        incident = []
        for i in range(k):
            incident.append(x[(i, k)])
        for j in range(k+1, n):
            incident.append(x[(k, j)])
        prob += (pulp.lpSum(incident) == 2), f"Degree_{k}"

    solver = pulp.PULP_CBC_CMD(msg=msg)

    def extract_edges_val() -> Dict[Tuple[int, int], float]:
        return {(i, j): x[(i, j)].value() or 0.0 for i in range(n) for j in range(i+1, n)}

    def components_from_edges(edges_on: Set[Tuple[int,int]]):
        """Return list of components (each as a list of nodes) from the undirected graph of 'on' edges."""
        adj = {k: [] for k in range(n)}
        for (i, j) in edges_on:
            adj[i].append(j)
            adj[j].append(i)
        seen = [False] * n
        comps = []
        for s in range(n):
            if not seen[s]:
                stack = [s]
                seen[s] = True
                comp = []
                while stack:
                    u = stack.pop()
                    comp.append(u)
                    for v in adj[u]:
                        if not seen[v]:
                            seen[v] = True
                            stack.append(v)
                comps.append(sorted(comp))
        return comps

    # Iteratively add subtour elimination constraints until single tour
    iteration = 0
    while True:
        iteration += 1
        status = prob.solve(solver)
        status_str = pulp.LpStatus.get(status, "Unknown")
        if status_str in ("Infeasible", "Undefined"):
            # No feasible solution under current cuts
            return status_str, math.inf, []

        # Edges selected (treat near-1 as on)
        xval = extract_edges_val()
        edges_on = {(i, j) for (i, j), v in xval.items() if v > 0.5 + tol}

        # If not enough edges are "on" (due to tolerance), include strong fractionals near 1
        if len(edges_on) < n:  # degree=2 implies exactly n edges in a tour
            edges_on |= {(i, j) for (i, j), v in xval.items() if v > 0.5}

        comps = components_from_edges(edges_on)

        # If exactly one component and it contains all nodes, we have a tour
        if len(comps) == 1 and len(comps[0]) == n:
            total_len = sum(D[i][j] * round(xval[(i, j)]) for i in range(n) for j in range(i+1, n))
            # Reconstruct tour from adjacency
            adj = {k: [] for k in range(n)}
            for (i, j) in edges_on:
                adj[i].append(j)
                adj[j].append(i)

            tour = [0]
            cur, prev = 0, None
            for _ in range(n-1):
                nxt = [v for v in adj[cur] if v != prev][0]
                tour.append(nxt)
                prev, cur = cur, nxt

            return "Optimal", total_len, tour

        # Otherwise, add a subtour elimination cut for each component with size < n
        added_cuts = 0
        for comp in comps:
            if 1 <= len(comp) < n:
                # SEC: sum_{i<j, i,j in S} x_ij <= |S| - 1
                pairs = []
                for idx_a in range(len(comp)):
                    for idx_b in range(idx_a + 1, len(comp)):
                        i, j = comp[idx_a], comp[idx_b]
                        a, b = (i, j) if i < j else (j, i)
                        pairs.append(x[(a, b)])
                if pairs:
                    cname = f"SEC_iter{iteration}_size{len(comp)}_{added_cuts}"
                    prob += (pulp.lpSum(pairs) <= len(comp) - 1), cname
                    added_cuts += 1

        if added_cuts == 0:
            # Shouldn't happen often; means solution already a single component but not a simple cycle
            # (numerical oddity). Force a small tightening by cutting the most “full” small subset.
            # As a fallback, break to avoid infinite loop.
            break

    # Fallback return if loop breaks unexpectedly
    return pulp.LpStatus[prob.status], float(pulp.value(prob.objective)), []

CITIES = [
    (0,0),(1,5),(5,2),(6,6),(8,3),(2,1),(7,8),(3,7),(9,4),(4,3)
]

status, length, tour = tsp_undirected_subtour_elimination(CITIES, msg=False)
print("Status:", status)
print("Tour length:", round(length, 4))
print("Tour (start at 0):", tour + [tour[0]] if tour else tour)

Status: Optimal
Tour length: 28.8531
Tour (start at 0): [0, 1, 7, 3, 6, 8, 4, 2, 9, 5, 0]
